## Grupo 7

## Análise Exploratória dos Dados

Neste notebook é feita uma análise detalhada dos dados de consumo de energia residencial em Londres durante o ano de 2013. O objetivo desta análise é compreender como diferentes fatores, como condições climáticas e características socioeconómicas, influenciam o uso de energia nas residências. Esta análise serve como base fundamental para a subsquente aplicação do algoritmo K-means. Ao compreender como os fatores mencionados afetam o consumo de energia, pode-se identificar variáveis relevantes e padrões de comportamento que são essenciais para um clustering eficaz.

### Inicialização da Sessão Spark

In [37]:
# Create our Spark session

import findspark
from pyspark.sql import SparkSession
import pyspark
from pyspark.sql import SparkSession, Row
import pyspark.sql.functions as F
from pyspark.sql.functions import col, avg
from pyspark.sql.functions import year, month, dayofmonth, dayofweek, hour, date_format
import matplotlib.pyplot as plt
from pyspark.sql.functions import corr
import pandas as pd
from pyspark.sql.functions import min, max

# Initialize Spark session
spark = SparkSession\
        .builder\
        .appName("TrabalhoFinal")\
        .config("spark.sql.shuffle.partitions",6)\
        .config("spark.sql.repl.eagereval.enabled",True)\
        .getOrCreate()

### Carregamento e visualização dos dados

In [38]:
# Carregar o arquivo Parquet num DataFrame
df = spark.read.parquet('dados_final_parquet')

# Mostrar as primeiras 5 linhas do DataFrame
df.show(5)

# Mostrar o esquema do DataFrame
df.printSchema()

+---------+-------------------+--------------+--------+-------+-------------+-----------+-------------------+----------+--------+-------------+-------------+-------+-----+---+----+
|    LCLid|               time|energy(kWh/hh)|stdorToU|  Acorn|Acorn_grouped|temperature|apparentTemperature|precipType|humidity|      summary|period_of_day|weekend|month|day|hour|
+---------+-------------------+--------------+--------+-------+-------------+-----------+-------------------+----------+--------+-------------+-------------+-------+-----+---+----+
|MAC000071|2013-01-01 00:00:00|        0.113 |     ToU|ACORN-J|  Comfortable|       7.01|               3.66|      rain|    0.73|Partly Cloudy|        Night|      0|    1|  1|   0|
|MAC000071|2013-01-01 00:30:00|        0.062 |     ToU|ACORN-J|  Comfortable|       7.01|               3.66|      rain|    0.73|Partly Cloudy|        Night|      0|    1|  1|   0|
|MAC000071|2013-01-01 01:00:00|        0.107 |     ToU|ACORN-J|  Comfortable|       7.49|      

In [39]:
# Contar o número total de linhas
row_count = df.count()
print(f"Número total de linhas: {row_count}")

# Contar o número total de colunas
column_count = len(df.columns)
print(f"Número total de colunas: {column_count}")

Número total de linhas: 93024229
Número total de colunas: 16


A visualização inicial dos dados revela que, tal como foi definido no notebook anterior, a base de dados contém 16 variáveis: `LCLid`(Identificador do consumidor), `time`(Data e hora da medição do consumo), `energy(kWh/hh)` (Consumo de energia), `stdorToU` (Tipo de tarifa) `Acorn` e `Acorn_grouped` (Classificação socioeconômica), `Temperature` (Temperatura), `apparentTemperatura` (Temperatura aparente), `precipType` (Tipo de precepitação), `Humidity` (Humidade), `Summary` (Resumo das condições climáticas), `Period_of_day` (Período do dia em que foi realizada a medição do consumo), `Weekend` (Se a medição do consumo foi realizada no fim de semana ou não), `Month` (Mês em que foi realizada a medição do consumo), `Day` (Dia em que foi realizada a medição do consumo), `Hour` (Hora em que foi realizada a medição do consumo).

Verifica-se também que o número de observações desta base de dados é 93024229.


### Conversão das variáveis categóricas

Nesta base de dados existem variáveis de diferentes tipos. As variáveis 'LCLid', 'stdorToU', 'Acorn', 'Acorn_grouped', 'precipType', 'Summary' e 'Period_of_day' são categóricas, (do tipo string); as variáveis 'energy(kWh/hh)', 'Temperature', 'apparentTemperatura' e 'Humidity' são numéricas (do tipo float); as variáveis 'Weekend', 'Month', 'Day' e 'Hour' são inteiras (do tipo integer), e, por fim, 'time' é uma variável temporal (do tipo timestamp).

As variáveis mencionadas como sendo numéricas, estão apresentadas incorretamente na base de dados como sendo do tipo string, em vez de o tipo float. Assim, antes de se prosseguir para a análise discritiva das variáveis, começou-se por converter as quatro variáveis categóricas para o tipo correto.

In [40]:
# Converter a coluna 'energy(kWh/hh)' para float
df = df.withColumn("energy(kWh/hh)", col("energy(kWh/hh)").cast("float"))

# Converter a coluna 'temperature' para float
df = df.withColumn("temperature", col("temperature").cast("float"))

# Converter a coluna 'apparentTemperature' para float
df = df.withColumn("apparentTemperature", col("apparentTemperature").cast("float"))

# Converter a coluna 'humidity' para float
df = df.withColumn("humidity", col("humidity").cast("float"))

df.printSchema()

root
 |-- LCLid: string (nullable = true)
 |-- time: timestamp (nullable = true)
 |-- energy(kWh/hh): float (nullable = true)
 |-- stdorToU: string (nullable = true)
 |-- Acorn: string (nullable = true)
 |-- Acorn_grouped: string (nullable = true)
 |-- temperature: float (nullable = true)
 |-- apparentTemperature: float (nullable = true)
 |-- precipType: string (nullable = true)
 |-- humidity: float (nullable = true)
 |-- summary: string (nullable = true)
 |-- period_of_day: string (nullable = true)
 |-- weekend: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- hour: integer (nullable = true)



### Variável `time`:

#### Extração de componentes principais

Extraíram-se os componentes temporais da coluna 'time', de forma a ser possível analisar o consumo de energia em diferentes escalas de tempo, como anual, mensal, semanal e horária. Esta extração é fundamental para identificar variações sazonais e diárias no consumo de energia, facilitando a análise de tendências e padrões de comportamento dos consumidores.

In [41]:
# Mostrar as primeiras linhas para verificar a coluna 'time'
df.select("time").show(5, truncate=False)

# Extração dos componentes temporais
df = df.withColumn("year", year("time")) \
       .withColumn("month", month("time")) \
       .withColumn("day", dayofmonth("time")) \
       .withColumn("day_of_week", dayofweek("time")) \
       .withColumn("hour", hour("time"))

# Mostrar as primeiras linhas com os componentes extraídos
df.select("time", "year", "month", "day", "day_of_week", "hour", "energy(kWh/hh)").show(5, truncate=False)

+-------------------+
|time               |
+-------------------+
|2013-01-01 00:00:00|
|2013-01-01 00:30:00|
|2013-01-01 01:00:00|
|2013-01-01 01:30:00|
|2013-01-01 02:00:00|
+-------------------+
only showing top 5 rows

+-------------------+----+-----+---+-----------+----+--------------+
|time               |year|month|day|day_of_week|hour|energy(kWh/hh)|
+-------------------+----+-----+---+-----------+----+--------------+
|2013-01-01 00:00:00|2013|1    |1  |3          |0   |0.113         |
|2013-01-01 00:30:00|2013|1    |1  |3          |0   |0.062         |
|2013-01-01 01:00:00|2013|1    |1  |3          |1   |0.107         |
|2013-01-01 01:30:00|2013|1    |1  |3          |1   |0.069         |
|2013-01-01 02:00:00|2013|1    |1  |3          |2   |0.102         |
+-------------------+----+-----+---+-----------+----+--------------+
only showing top 5 rows



#### Análise da distribuição temporal 

#####  Contagem de registos por ano:

In [42]:
# Contagem de registos por ano
df.groupBy("year").count().orderBy("year").show()

+----+--------+
|year|   count|
+----+--------+
|2013|93024229|
+----+--------+



Uma vez que apenas temos dados de 2013, ao verificar a quantidade de registos de cada ano, observa-se que todos os 93024229 registos de consumo de energia pertencem ao ano de 2013. Isto confirma que se está a trabalhar exclusivamente com dados de um único ano, proporcionando uma análise focada desse período específico.

##### Contagem de registos por mês:

In [43]:
# Contagem de registos por mês
df.groupBy("year", "month").count().orderBy("year", "month").show()

+----+-----+-------+
|year|month|  count|
+----+-----+-------+
|2013|    1|8184186|
|2013|    2|7350033|
|2013|    3|8064337|
|2013|    4|7758188|
|2013|    5|7976841|
|2013|    6|7683709|
|2013|    7|7890224|
|2013|    8|7831151|
|2013|    9|7524276|
|2013|   10|7728268|
|2013|   11|7434310|
|2013|   12|7598706|
+----+-----+-------+



A análise da contagem de registos por mês revela que todos os meses de 2013 possuem uma quantidade significativa de dados, o que indica que não há períodos de ausência de informações no conjunto de dados. O mês de janeiro possui o maior número de registos, 8184186, enquanto o mês de fevereiro apresenta o menor número, 7350033. 


##### Contagem de registos por dia da semana:

In [44]:
# Contagem de registros por dia da semana
df.groupBy("day_of_week").count().orderBy("day_of_week").show()

+-----------+--------+
|day_of_week|   count|
+-----------+--------+
|          1|13250305|
|          2|13246347|
|          3|13508329|
|          4|13260551|
|          5|13255193|
|          6|13251619|
|          7|13251885|
+-----------+--------+



A análise da contagem de registos por dia da semana confirma que os dados de consumo de energia foram recolhidos de forma consistente durante todos os dias, sem interrupções significativas. A uniformidade dos dados ao longo dos dias da semana também sugere que não há variações significativas em dias específicos, permitindo uma análise equilibrada do comportamento de consumo.

##### Contagem de registos por hora do dia:

In [45]:
# Contagem de registros por hora do dia
df.groupBy("hour").count().orderBy("hour").show(24, truncate=False)

+----+-------+
|hour|count  |
+----+-------+
|0   |3876439|
|1   |3876078|
|2   |3876320|
|3   |3876438|
|4   |3876282|
|5   |3876187|
|6   |3876293|
|7   |3876328|
|8   |3876006|
|9   |3875842|
|10  |3875742|
|11  |3875930|
|12  |3875435|
|13  |3875839|
|14  |3875578|
|15  |3875893|
|16  |3875949|
|17  |3876098|
|18  |3876101|
|19  |3875893|
|20  |3875783|
|21  |3875770|
|22  |3875965|
|23  |3876040|
+----+-------+



Os dados mostram que a contagem de registos é consiste ao longo das 24 horas do dia, sem grandes variações que indiquem interrupções ou falhas significativas na recolha de dados em qualquer hora específica. Cada hora do dia possui aproximadamente 3,876 milhões de registos, indicando uma distribuição uniforme dos dados de consumo de energia ao longo do dia.

### Variáveis  `time` e `energy(kWh/hh)`:

#### Média de energia consumida por mês ao longo do ano:

In [46]:
# Análise de tendências: Média de energia consumida por mês ao longo do ano
df.groupBy("year", "month").agg(F.avg("energy(kWh/hh)").alias("avg_energy")).orderBy("year", "month").show()

+----+-----+-------------------+
|year|month|         avg_energy|
+----+-----+-------------------+
|2013|    1| 0.2634128334789319|
|2013|    2|0.25826842914422393|
|2013|    3|0.25230816109227927|
|2013|    4|0.20975695773926714|
|2013|    5|0.18549650490986722|
|2013|    6| 0.1740460592293063|
|2013|    7|0.16748300254036894|
|2013|    8| 0.1641081567347799|
|2013|    9|0.18269060815285817|
|2013|   10|0.19814050975992828|
|2013|   11|0.23071061652838706|
|2013|   12|0.24410824271802928|
+----+-----+-------------------+



A análise do consumo médio de energia por mês ao longo do ano evidencia que o consumo de energia é sazonal, com maiores valores nos meses mais frios e menores nos meses mais quentes. Os valores elevados de consumo de energia em meses mais frios confirmam a expectativa de que as temperaturas mais baixas levam a um aumento na demanda por energia, especialmente para aquecimento. Ao mesmo tempo, os valores reduzidos de consumo de energia nos meses mais quentes, devem-se à menor demanda por aquecimento e, possivelmente, ao uso reduzido de iluminação, uma vez que os dias são mais longos.


#### Média de energia consumida por dia da semana:

In [47]:
# Análise de sazonalidade: Média de energia consumida por dia da semana
df.groupBy("day_of_week").agg(F.avg("energy(kWh/hh)").alias("avg_energy")).orderBy("day_of_week").show()

+-----------+-------------------+
|day_of_week|         avg_energy|
+-----------+-------------------+
|          1|0.22193239915116147|
|          2|0.21017402187146156|
|          3|0.20847254002258206|
|          4|0.20763971303627426|
|          5|0.20705048475367155|
|          6|0.20771408447118617|
|          7|0.21313619972477293|
+-----------+-------------------+



A análise sugere que o dia da semana não tem um impacto significativo no consumo médio de energia. Ainda assim, apesar de pequenas, existem diferenças entre o consumo de energia no fim de semana e nos dias úteis, possivelmente devido ao aumento das atividades domésticas e tempo gasto em casa ao sábado e ao domingo. Mas estas são diferenças leves, o que leva à conclusão de que, ao longo da semana, os padrões de consumo de energia são relativamente consistentes. Essa uniformidade no consumo sugere que não há necessidade de ajustar a oferta de energia baseada no dia da semana, o que simplifica a gestão de energia.

#### Média de energia consumida por hora do dia:

In [48]:
# Análise de sazonalidade: Média de energia consumida por hora do dia
df.groupBy("hour").agg(F.avg("energy(kWh/hh)").alias("avg_energy")).orderBy("hour").show(24, truncate=False)

+----+-------------------+
|hour|avg_energy         |
+----+-------------------+
|0   |0.17461908090909242|
|1   |0.14128488344100087|
|2   |0.12313638204991473|
|3   |0.11489625823720466|
|4   |0.11461191587186276|
|5   |0.12618961738717482|
|6   |0.15774100678389213|
|7   |0.19213838337910766|
|8   |0.21334765253864238|
|9   |0.2157494438027396 |
|10  |0.21373026695166367|
|11  |0.21421654063638995|
|12  |0.21635153026491782|
|13  |0.21431271782499334|
|14  |0.21047962093618497|
|15  |0.21590568374263505|
|16  |0.2404508837176129 |
|17  |0.27840272619943496|
|18  |0.3081346652626227 |
|19  |0.3180154375131287 |
|20  |0.3111478179234978 |
|21  |0.29277097665082347|
|22  |0.253281202033758  |
|23  |0.19994560358189528|
+----+-------------------+



A análise da média de energia consumida por hora do dia revela que o consumo energético é menor durante a madrugada e aumenta significativamente a partir do final da tarde, atingindo um pico entre as 17h e as 21h. Estes padrões são coerentes com o comportamento típico de uma residência, onde o uso de energia é menor quando os moradores estão a dormir, e aumenta quando estão ativos em casa, especialmente durante as refeições e atividades domésticas.

#### Análise dos dias com maior consumo de energia:

In [49]:
# Análise de picos e vales: Encontrar os dias com maior consumo de energia
df.groupBy(F.date_format("time", "yyyy-MM-dd").alias("day")).agg(F.sum("energy(kWh/hh)").alias("total_energy")).orderBy(F.desc("total_energy")).limit(20).show()

+----------+-----------------+
|       day|     total_energy|
+----------+-----------------+
|2013-01-20|82650.49201568787|
|2013-01-19|80900.96302181052|
|2013-01-18|77294.25401508482|
|2013-03-24|76861.65902009164|
|2013-02-24| 76489.9490114185|
|2013-01-13|75265.22401094134|
|2013-01-21|75034.14600998152|
|2013-02-10|74558.53702018224|
|2013-01-22| 74398.3420162173|
|2013-01-25|73746.75901319284|
|2013-01-24| 73479.5180237625|
|2013-01-23|73407.77902140608|
|2013-01-17|73059.33601903182|
|2013-02-03|73004.99001688662|
|2013-03-23|72746.36001087748|
|2013-01-14|72611.05001372832|
|2013-01-16|72377.18301073939|
|2013-02-23|72307.56701906107|
|2013-01-12|71923.91802272899|
|2013-02-11|71613.77302433737|
+----------+-----------------+



A análise dos dias com maior consumo de energia revela que os picos de consumo ocorreram predominantemente durante os meses de inverno, especialmente em janeiro, seguido por dias de fevereiro e março, que são tradicionalmente meses com temperaturas mais baixas. Este padrão indica uma tendência clara de aumento do consumo energético durante períodos mais frios, possivelmente devido à maior necessidade de aquecimento. 
Estes resultados são cruciais para entender os fatores sazonais que influenciam o consumo de energia, permitindo planear e gerenciar melhor a demanda energética em função das variações climáticas.

#### Análise dos dias com menor consumo de energia:

In [50]:
# Análise de picos e vales: Encontrar os dias com menor consumo de energia
df.groupBy(F.date_format("time", "yyyy-MM-dd").alias("day")).agg(F.sum("energy(kWh/hh)").alias("total_energy")).orderBy(F.asc("total_energy")).limit(20).show()

+----------+------------------+
|       day|      total_energy|
+----------+------------------+
|2013-08-07| 40310.15602253727|
|2013-08-20|40333.487019924796|
|2013-08-21| 40445.90202320507|
|2013-08-08| 40482.32302581682|
|2013-08-09| 40622.97302545176|
|2013-08-10| 40681.17003072298|
|2013-08-29|40691.384026773274|
|2013-08-14| 40704.04902218224|
|2013-08-06|40768.549024426495|
|2013-08-13|40824.086022065254|
|2013-08-03|40935.213019034825|
|2013-08-28| 41020.24502353044|
|2013-08-12| 41075.69402037235|
|2013-08-23|41079.668025292456|
|2013-08-19|41085.040020832676|
|2013-08-15| 41179.76002305315|
|2013-08-30|  41254.7220249573|
|2013-07-26| 41272.61602753389|
|2013-07-11| 41310.23002905073|
|2013-08-27| 41429.71902396914|
+----------+------------------+



A análise dos dias com menor consumo de energia evidencia que a maioria dos dias com menor uso energético ocorreram em agosto, um mês tipicamente quente em Londres. Dos 20 dias de menor consumo listados, 18 pertencem a agosto, refletindo a redução na necessidade de aquecimento durante o verão. Os outros dois dias de menor consumo são de julho, outro mês de temperaturas elevadas. Isto confirma que o consumo de energia é significativamente menor nos períodos de clima mais quente, quando a demanda por aquecimento é reduzida. Mais uma vez, esta informação é crucial para entender os padrões sazonais de consumo e ajustar estratégias de gestão energética conforme as variações climáticas anuais.

### Variável `stdorToU`:

In [51]:
# Obter e listar os valores distintos na coluna "stdorToU"
print([row["stdorToU"] for row in df.select("stdorToU").distinct().collect()])

['Std', 'ToU']


A variável 'stdorToU' refere-se ao tipo de tarifa de energia aplicada aos consumidores e está dividida em duas categorias principais: 'Std' (Standard Tariffs) e 'ToU' (Time Of Use Pricing). A categoria 'Std' representa as tarifas padrão, onde o custo da energia é fixo e não varia ao longo do dia. Já a categoria 'ToU' aplica uma estrutura de preços que varia conforme a hora do dia, incentivando os consumidores a reduzir o uso de energia durante os períodos de alta demanda e a aproveitar os períodos de menor custo para suas atividades de consumo energético.

#### Consumo de energia para cada tipo de tarifa:

In [52]:
# Calcular a média, minimo e maximo de consumo de energia para cada valor de "stdorToU"
df.groupBy("stdorToU").agg(avg("energy(kWh/hh)").alias("avg_energy"), min("energy(kWh/hh)").alias("min_energy"), max("energy(kWh/hh)").alias("max_energy")).show()

+--------+-------------------+----------+----------+
|stdorToU|         avg_energy|min_energy|max_energy|
+--------+-------------------+----------+----------+
|     Std|0.21445467223957645|       0.0|    10.761|
|     ToU| 0.1967772729993727|       0.0|     7.567|
+--------+-------------------+----------+----------+



Ao calcular a média, o mínimo e o máximo de consumo de energia para cada tipo de tarifa, observa-se que consumidores que utilizam a tarifa padrão (Std) tendem a ter um consumo médio de energia mais alto (0.214 kWh/hh) em comparação com aqueles que utilizam a tarifa baseada no tempo de uso (ToU), que apresentam um consumo médio de 0.197 kWh/hh. Essa diferença pode ser explicada pelo facto de que os consumidores com tarifas ToU possivelmente ajustam os seus hábitos de consumo para economizar energia durante os períodos de tarifa mais alta, aproveitando os momentos de tarifa mais baixa para usar mais energia. Por outro lado, os consumidores com tarifas padrão não têm essa variação tarifária e, portanto, não alteram significativamente os seus padrões de consumo ao longo do dia.

### Variável `Acorn`:

In [53]:
df.select("Acorn").distinct().count()


19

A variável 'Acorn' refere-se à classificação ACORN (A Classification of Residential Neighborhoods), um sistema utilizado para segmentar as áreas residenciais do Reino Unido em diferentes tipos socioeconómicos. Esta classificação utiliza diversos dados demográficos, sociais e económicos para entender e analisar padrões de consumo, comportamentos de mercado e necessidades sociais de diferentes grupos populacionais.
A análise revela que os dados de consumo energético estão distribuídos em 19 categorias distintas da classificação ACORN. Cada uma dessas categorias representa um perfil socioeconómico específico, com características demográficas e comportamentais que influenciam o uso de energia. 

#### Consumo de energia para cada tipo de Acorn:

In [54]:
# Calcular a média, minimo e maximo de consumo de energia para cada valor de "Acorn"
df.groupBy("Acorn").agg(avg("energy(kWh/hh)").alias("avg_energy"), min("energy(kWh/hh)").alias("min_energy"), max("energy(kWh/hh)").alias("max_energy")).orderBy("avg_energy", ascending=False).show()

+-------+-------------------+----------+----------+
|  Acorn|         avg_energy|min_energy|max_energy|
+-------+-------------------+----------+----------+
|ACORN-A|0.39236076917502455|       0.0|     9.141|
|ACORN-D|0.28058464655858584|       0.0|     7.567|
|ACORN-B|  0.250594707519044|       0.0|     3.944|
|ACORN-C|0.24588035702967423|       0.0|     5.135|
|ACORN-U| 0.2447749315365297|       0.0|      8.75|
|ACORN-J|0.23703345313832513|       0.0|     6.452|
|ACORN-H|0.22883769335401252|       0.0|     5.647|
|ACORN-E| 0.2156560991828898|       0.0|    10.761|
|ACORN-K|0.21082325851511968|       0.0|     5.812|
|ACORN-G|0.20897516117141274|       0.0|     5.943|
|ACORN-M|0.20828758107588516|       0.0|     5.308|
|ACORN-L|0.20734710895340838|       0.0|     6.234|
|ACORN-I|0.19968903868650384|       0.0|     5.361|
|ACORN-F|0.19256254087952868|       0.0|     8.875|
|ACORN-N|0.18969790406127335|       0.0|     6.299|
|ACORN-O|0.17609069361380272|       0.0|     5.778|
|ACORN-Q|0.1

Observando os resultados, verifica-se que há uma variação considerável no consumo médio de energia entre as diferentes categorias ACORN. Por exemplo, a categoria "ACORN-A" apresenta o maior consumo médio de energia (0,392 kWh/hh), enquanto a categoria "ACORN-P" tem o menor consumo médio (0,134 kWh/hh). Estas diferenças podem refletir variações nas condições económicas e nos hábitos de consumo das diferentes áreas residenciais.

A análise da tabela mostra também que, além das diferenças médias, o consumo de energia mínima e máxima varia significativamente entre as categorias, o que pode indicar diferentes níveis de eficiência energética e comportamentos de uso em diferentes grupos socioeconómicos.

Assim, a categoria "ACORN-A" poderá representar áreas com alta renda e padrões de vida elevados, enquanto "ACORN-P" deve representar áreas de baixa renda e com menos recursos.
O maior consumo médio de energia na categoria "ACORN-A"  pode indicar que essas residências possuem mais dispositivos eletrónicos, sistemas de aquecimento ou ar-condicionado, além de uma maior capacidade económica dos moradores. Categorias como "ACORN-P", que têm um consumo médio mais baixo, podem estar limitadas economicamente e, portanto, mais incentivadas a adotar medidas de economia de energia e eficiência para reduzir os custos.

### Variável `Temperature`:

In [55]:
df.describe("temperature").show()
df.select(corr("temperature", "energy(kWh/hh)")).first()[0]

+-------+------------------+
|summary|       temperature|
+-------+------------------+
|  count|          93003278|
|   mean| 10.75452082482803|
| stddev|6.4664309086696985|
|    min|             -3.86|
|    max|              32.4|
+-------+------------------+



-0.08175219650545068

Ao analisar a variável 'Temperature' conclui-se que a temperatura média durante o período analisado foi de aproximadamente 10.75°C, com um desvio padrão de cerca de 6.47°C, o que indica variações climáticas significativas ao longo do tempo. Os registos mostram uma amplitude de temperaturas, com a mínima sendo -3.86°C e a máxima atingindo 32.4°C, o que reflete a presença de dias tanto frios quanto quentes no conjunto de dados.

A correlação entre a temperatura e o consumo de energia ('energy(kWh/hh)') é de cerca de -0.0818, sugerindo uma relação negativa fraca. Isso significa que, à medida que a temperatura aumenta, o consumo de energia tende a diminuir ligeiramente. Este resultado é esperado em muitas regiões urbanas, onde o aumento da temperatura pode reduzir a necessidade de aquecimento, uma das principais causas do consumo elevado de energia nos meses mais frios. Em dias mais quentes, há uma tendência de menor consumo de energia, possivelmente porque a necessidade de aquecimento é reduzida e a demanda por energia para outras finalidades não aumenta proporcionalmente.


### Variável `ApparentTemperature`:

In [56]:
df.describe("apparentTemperature").show()
df.select(corr("apparentTemperature", "energy(kWh/hh)")).first()[0]

+-------+-------------------+
|summary|apparentTemperature|
+-------+-------------------+
|  count|           93003278|
|   mean|  9.516789506188335|
| stddev|  7.755332473471355|
|    min|              -8.88|
|    max|              32.42|
+-------+-------------------+



-0.08715967753121938

Tal como foi feito para a variável 'Temperature', analisa-se a variável 'ApparentTemperature' da mesma forma. 

A sensação térmica média é de aproximadamente 9.52°C com um desvio padrão de 7.75°C, indicando uma variação significativa nas sensações térmicas dos consumidores ao longo do tempo. A amplitude de valores observados vai de -8.88°C a 32.42°C, cobrindo uma variedade de sensações térmicas, desde muito frio até bastante calor.

A correlação entre a sensação térmica e o consumo de energia é de cerca de -0.0872, o que, assim como na análise da temperatura real, sugere uma correlação negativa fraca. Isso implica que, à medida que a sensação térmica aumenta, o consumo de energia tende a diminuir ligeiramente. Esse resultado é consistente com a análise da temperatura real, reforçando a ideia de que em dias mais quentes, seja pela temperatura real ou pela sensação térmica, há uma tendência de menor consumo de energia, possivelmente devido à redução na necessidade de aquecimento.

Assim, tanto a temperatura real quanto a sensação térmica têm uma relação inversa com o consumo de energia, indicando que os padrões de uso energético são sensíveis às variações térmicas. Esta análise é crucial para entender como diferentes fatores climáticos influenciam o consumo de energia, ajudando a identificar períodos de maior ou menor demanda e permitindo a criação de estratégias de gestão de energia mais eficientes.

#### Relação entre consumo de energia e variação de temperatura ao longo do ano:

In [57]:
df.groupBy("year", "month").agg(avg("energy(kWh/hh)").alias("avg_energy"), avg("temperature").alias("avg_temperature"), avg("apparentTemperature").alias("avg_apparentTemperature")).orderBy("year", "month").show()

+----+-----+-------------------+------------------+-----------------------+
|year|month|         avg_energy|   avg_temperature|avg_apparentTemperature|
+----+-----+-------------------+------------------+-----------------------+
|2013|    1| 0.2634128334789319| 4.507099706819547|       2.15118046473127|
|2013|    2|0.25826842914422393|3.8019676064091454|     0.6398844497025731|
|2013|    3|0.25230816109227927|3.8360146752132245|     0.7365836941708909|
|2013|    4|0.20975695773926714| 8.621065225162132|       6.98826364869023|
|2013|    5|0.18549650490986722|11.656647118123619|     11.043085345297113|
|2013|    6| 0.1740460592293063|15.100912745078194|     15.021999989885575|
|2013|    7|0.16748300254036894|20.036475926553223|     20.072979640197318|
|2013|    8| 0.1641081567347799|18.503450470064706|      18.54798906872595|
|2013|    9|0.18269060815285817|14.809744714123864|      14.71806693741399|
|2013|   10|0.19814050975992828| 13.34300803538098|     13.004571316985606|
|2013|   11|

A análise mostra a média de energia consumida, a média de temperatura e a média da sensação térmica por mês ao longo do ano de 2013. Observa-se que nos meses mais quentes, como julho e agosto, a média de temperatura é significativamente mais alta, e o consumo médio de energia é menor. Isso pode ser atribuído à menor necessidade de aquecimento durante esses meses. Por outro lado, nos meses mais frios, como janeiro e fevereiro, a temperatura média é baixa, e o consumo médio de energia é mais elevado, refletindo a maior necessidade de aquecimento residencial. Essas tendências indicam que o consumo de energia está inversamente relacionado à temperatura: quanto maior a temperatura, menor o consumo de energia, e vice-versa.

### Variável `Humidity`:

In [58]:
df.describe("humidity").show()

+-------+------------------+
|summary|          humidity|
+-------+------------------+
|  count|          93003278|
|   mean|   0.7662578863007|
| stddev|0.1465263901502372|
|    min|              0.23|
|    max|               1.0|
+-------+------------------+



A análise da variável 'Humidity' revelou que a média de humidade registada é de aproximadamente 76,63%, com um desvio padrão de 14,65%. O valor mínimo de humidade é de 23% e o valor máximo é de 100%.

#### Relação entre consumo de energia e variação de humidade ao longo do ano:

In [59]:
df.groupBy("year", "month").agg(avg("energy(kWh/hh)").alias("avg_energy"), avg("humidity").alias("avg_humidity")).orderBy("year", "month").show()

+----+-----+-------------------+------------------+
|year|month|         avg_energy|      avg_humidity|
+----+-----+-------------------+------------------+
|2013|    1| 0.2634128334789319|0.8603423135873728|
|2013|    2|0.25826842914422393|0.7775175853221586|
|2013|    3|0.25230816109227927|0.7703030955773658|
|2013|    4|0.20975695773926714|0.7003188577984952|
|2013|    5|0.18549650490986722|0.7052870757772086|
|2013|    6| 0.1740460592293063|0.7097439154406063|
|2013|    7|0.16748300254036894|0.6689669838462001|
|2013|    8| 0.1641081567347799|0.6993439456003219|
|2013|    9|0.18269060815285817|0.8058840143757786|
|2013|   10|0.19814050975992828|0.8503317248355393|
|2013|   11|0.23071061652838706|0.8202744204068072|
|2013|   12|0.24410824271802928|0.8307142369731103|
+----+-----+-------------------+------------------+



Na análise da média de energia consumida por mês, comparada com a média de humidade, observa-se que há uma tendência de maior consumo de energia nos meses de maior humidade. Por exemplo, janeiro e dezembro, que possuem médias de humidade relativamente altas, apresentam também maiores consumos médios de energia. Isso pode indicar que em condições de maior humidade, a demanda por energia, possivelmente para aquecimento ou desumidificação, aumenta.

Assim, decide-se testar a hipótese de que uma maior humidade estaria associada a uma menor temperatura e vice-versa. Para tal, é importante analisar a relação entre os dados de temperatura e os dados de humidade, para verificar se há uma correlação inversa entre essas duas variáveis.

In [60]:
df.select(corr("humidity", "temperature")).first()[0]

-0.4024816524132465

O valor da correlação obtida foi de -0.4025, indicando uma correlação negativa significativa entre as duas variáveis. Isso confirma que, de facto, quando a humidade aumenta, a temperatura tende a diminuir e, inversamente, quando a humidade é menor, a temperatura é mais alta. Essa relação inversa é consistente com a hipótese formulada e reforça a ideia de que os níveis de humidade estão inversamente relacionados às variações de temperatura, influenciando, portanto, o consumo de energia para aquecimento ou arrefecimento das residências.

### Variável `PrecipType`:

In [61]:
# Obter e listar os valores distintos na coluna "precipType"
print([row["precipType"] for row in df.select("precipType").distinct().collect()])

['snow', 'rain', None]


In [62]:
# Calcular a média, minimo e maximo de consumo de energia para cada valor de "precipType"
df.groupBy("precipType").agg(avg("energy(kWh/hh)").alias("avg_energy"), min("energy(kWh/hh)").alias("min_energy"), max("energy(kWh/hh)").alias("max_energy")).show()

+----------+-------------------+----------+----------+
|precipType|         avg_energy|min_energy|max_energy|
+----------+-------------------+----------+----------+
|      NULL|0.13614414599067218|       0.0|     4.167|
|      snow| 0.2394616833989164|       0.0|     8.717|
|      rain|0.20946303667197225|       0.0|    10.761|
+----------+-------------------+----------+----------+



A análise da variável 'precipType' identifica três tipos distintos de precipitação: 'snow' (neve), 'rain' (chuva) e 'None' (ausência de precipitação). O consumo médio de energia varia conforme o tipo de precipitação presente. Observa-se que a categoria 'snow' está associada ao maior consumo médio de energia (0,239 kWh/hh), seguida pela categoria 'rain' (0,209 kWh/hh) e, por último, pela ausência de precipitação (0,136 kWh/hh).

Estas diferenças podem ser explicadas pela influência das condições meteorológicas no consumo de energia. Durante períodos de neve, que são tipicamente mais frios, há uma maior necessidade de aquecimento, o que resulta num consumo de energia mais elevado. Da mesma forma, a presença de chuva também está associada a temperaturas mais baixas e a um aumento no uso de energia, embora não tão significativo quanto durante a neve. Em contraste, quando não há precipitação, o consumo médio de energia é menor, refletindo condições provavelmente mais amenas.

Portanto, pode-se concluir que a precipitação, especialmente a neve, está relacionada a um aumento no consumo de energia devido à maior demanda por aquecimento durante períodos de tempo mais frio.

### Variável `Summary`

In [63]:
# Obter e listar os valores distintos na coluna "summary"
print([row["summary"] for row in df.select("summary").distinct().collect()])

['Clear', 'Foggy', 'Windy and Overcast', 'Windy and Mostly Cloudy', 'Partly Cloudy', 'Breezy and Overcast', 'Mostly Cloudy', 'Windy', 'Overcast', 'Breezy', 'Breezy and Partly Cloudy', 'Breezy and Mostly Cloudy', None]


In [64]:
# Calcular a média, minimo e maximo de consumo de energia para cada valor de "summary"
df.groupBy("summary").agg(avg("energy(kWh/hh)").alias("avg_energy"), min("energy(kWh/hh)").alias("min_energy"), max("energy(kWh/hh)").alias("max_energy")).orderBy("avg_energy", ascending=False).show()

+--------------------+-------------------+----------+----------+
|             summary|         avg_energy|min_energy|max_energy|
+--------------------+-------------------+----------+----------+
|  Windy and Overcast| 0.2984281739503341|       0.0|     5.508|
| Breezy and Overcast|0.24748868275776237|       0.0|     6.634|
|               Windy|0.24342413694267265|       0.0|     5.935|
|       Mostly Cloudy| 0.2372803879835382|       0.0|    10.386|
|Breezy and Partly...|0.23662438200752384|       0.0|     8.205|
|       Partly Cloudy| 0.2291968703552324|       0.0|    10.761|
|Breezy and Mostly...|0.21985860462547396|       0.0|    10.457|
|            Overcast| 0.2177100995468319|       0.0|      10.0|
|Windy and Mostly ...|0.21227334848386503|       0.0|    10.408|
|               Foggy|0.20089073655365725|       0.0|      9.65|
|              Breezy| 0.1936111289939444|       0.0|     9.001|
|               Clear|0.17189201507653193|       0.0|    10.528|
|                NULL|0.1

A variável summary categoriza as condições meteorológicas em diferentes tipos, como 'Clear' (limpo), 'Foggy' (nevoeiro), 'Windy and Overcast' (ventoso e encoberto), entre outros. A análise mostra a média, o mínimo e o máximo de consumo de energia para cada categoria de condição climática.

Observa-se que as condições com maior média de consumo de energia são 'Windy and Overcast' (ventoso e encoberto) e 'Breezy and Overcast' (com brisa e encoberto), com valores de 0,298 e 0,247 kWh/hh, respetivamente. Em contraste, as condições 'Clear' (limpo) e 'Breezy' (com brisa) têm os menores consumos médios de energia, com 0,171 e 0,193 kWh/hh, respetivamente.

Estas diferenças podem ser atribuídas ao facto de que condições meteorológicas adversas, como vento e céu encoberto, geralmente estão associadas a temperaturas mais baixas e, consequentemente, a um aumento na demanda por aquecimento. Por outro lado, em dias limpos ou com brisa, a temperatura tende a ser mais amena, reduzindo a necessidade de aquecimento e, portanto, o consumo de energia.

Portanto, conclui-se que as condições meteorológicas mais rigorosas levam a um aumento no consumo de energia, enquanto condições mais amenas estão associadas a um menor uso de energia.

### Conclusões finais

Após a análise detalhada dos dados de consumo de energia residencial em Londres, foram identificados diversos padrões e tendências significativas.

Primeiramente, foi observado que nos meses mais frios, como janeiro e dezembro, há um aumento substancial no consumo de energia. Este aumento é impulsionado pela maior necessidade de aquecimento durante os períodos de temperaturas baixas, o que eleva significativamente o uso de energia em comparação com os meses mais quentes.

Além disso, a análise revela que durante o período noturno, o consumo de energia é reduzido. Isto ocorre porque a maioria das atividades que exigem mais energia, como cozinhar e realizar tarefas domésticas, tendem a ser realizada durante o dia. Assim, o consumo de energia diminui à noite quando as pessoas estão a dormir ou menos ativas nas suas residências.

No entanto, entre as 17h e as 21h, verifica-se um pico de consumo de energia. Esse aumento é provavelmente devido ao facto de que entre essas horas, as pessoas voltam a casa depois do trabalho e utilizam mais aparelhos elétricos para atividades como cozinhar ou ver televisão.

Observou-se também que os usuários que optam por tarifas padrão (Std) apresentam um consumo médio de energia mais elevado em comparação com aqueles que utilizam a tarifa por uso (ToU). A tarifa por uso incentiva a economia de energia, promovendo o consumo em horários de menor custo e reduzindo o consumo em períodos de tarifas mais altas.

Em relação à temperatura, foi constatada uma correlação negativa com o consumo de energia. À medida que a temperatura diminui, o consumo de energia aumenta, refletindo a necessidade de aquecimento nos períodos mais frios. Esse comportamento é consistente com a maior demanda por aquecimento durante os meses de inverno.

A análise da humidade mostra que um aumento na humidade está associado a um aumento no consumo de energia. Isso pode ser explicado pelo uso de aparelhos de desumidificação ou aquecimento em condições de alta humidade, que contribuem para o aumento do consumo energético.

Finalmente, em dias com precipitação, seja em forma de neve ou chuva, o consumo de energia tende a ser mais alto. Isso indica que durante condições climáticas adversas, há uma maior necessidade de aquecimento, o que leva a um aumento no consumo de energia.

Estas observações fornecem uma compreensão detalhada de como diferentes fatores influenciam o consumo de energia residencial e podem ser utilizadas para informar políticas de eficiência energética, ajudando os consumidores a gerir melhor o seu uso de energia.